#notebooks.clustering_task

In [8]:
from sklearn.cluster import KMeans
import numpy as np
from functions.circuits_utilities import generate_random_circuit_from_sequence, measure_zero_probability

In [9]:
dna_sequences = ['GTAAA', 'GTTCCGCCA', 'CTGTCT', 'TGATGCACA', 'TACACACAGA',
                 'ACAATCG', 'GTCAATGGCC', 'CCA', 'GCGC', 'GCTTGACGG']

In [11]:
probabilities = []
for sequence in dna_sequences:
    qc = generate_random_circuit_from_sequence(sequence)
    p_zero = measure_zero_probability(qc)
    probabilities.append(p_zero)

# Perform Clustering using k-means clustering with the measured probabilities
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(np.array(probabilities).reshape(-1, 1))
for sequence, cluster in zip(dna_sequences, clusters):
    print(f"Sequence: {sequence}, Cluster: {cluster}")

Sequence: GTAAA, Cluster: 2
Sequence: GTTCCGCCA, Cluster: 1
Sequence: CTGTCT, Cluster: 0
Sequence: TGATGCACA, Cluster: 1
Sequence: TACACACAGA, Cluster: 2
Sequence: ACAATCG, Cluster: 1
Sequence: GTCAATGGCC, Cluster: 1
Sequence: CCA, Cluster: 2
Sequence: GCGC, Cluster: 0
Sequence: GCTTGACGG, Cluster: 0
